In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import glob
import collections
import numpy as np
from tqdm import tqdm
from PIL import Image

In [ ]:
os.chdir('gdrive/My Drive/hotdog_data')

In [ ]:
seed = 42

In [4]:
image_list = []
for filename in tqdm(glob.glob('train/hotdog/*.jpg')):
    im = Image.open(filename).convert('RGB')  # in case some are grayscale
    image_list.append((im, 1))

100%|██████████| 708/708 [00:03<00:00, 212.23it/s]


In [5]:
for filename in tqdm(glob.glob('train/not_hotdog/*.jpg')):
    im = Image.open(filename).convert('RGB')
    image_list.append((im, 0))

100%|██████████| 963/963 [00:04<00:00, 204.11it/s]


In [ ]:
image_list = [(item[0].rotate(90, expand=True),item[1]) if item[0].size[0]<item[0].size[1] else item for item in image_list]
image_list = [(item[0].resize((224,224)),item[1]) for item in image_list]  # нужный размер

In [ ]:
X_train = [np.array(item[0]) for item in image_list]
y_train = [item[1] for item in image_list]

# Задание 3. VGG.

In [ ]:
y_train = np.array(y_train).reshape(1671,1)

In [ ]:
X_train = [image.reshape(1, 224, 224, 3) for image in X_train]

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model

In [ ]:
X_train = [preprocess_input(image) for image in X_train]

In [12]:
modelvgg = VGG16(weights='imagenet')
modelvgg.layers.pop()
modelvgg = Model(inputs=modelvgg.inputs, outputs=modelvgg.layers[-2].output)
modelvgg.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
X_train_vgg = []

In [14]:
for image in tqdm(X_train):
    X_train_vgg.append(modelvgg.predict(image).flatten())

100%|██████████| 1671/1671 [00:47<00:00, 35.15it/s]


In [ ]:
X_train = np.array(X_train_vgg)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

In [19]:
X_train.shape, y_train.shape

((1336, 4096), (1336, 1))

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
linear_classifier = LinearSVC(random_state=seed, tol=1e-5)

In [22]:
linear_classifier.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=42, tol=1e-05,
          verbose=0)

In [24]:
linear_classifier.score(X_test, y_test)

0.9731343283582089

Линейный SVC на фичах из VGG16 показал точность в 97.3% (ошибка всего в ~0.027). Высокий результат ожидаем, поскольку ImageNet - один из классических (сейчас) подходов к классификации изображений.

Важно заметить, что SVM, а также случайный лес и gradient boosting trees, хотя и обучаются на RGB-векторах ОЧЕНЬ долго из за их огромной длины, показывают результат гораздо хуже неоткалиброванной CNN.

Результат свёрточной нейронной сети гораздо лучше SVM даже при небольшой выборке, однако у нас будет слишком мало данных, чтобы превзойти ImageNet, даже при аугментации. Результат, безусловно, можно было бы улучшить подбором параметров, различными техниками нормализации, и т.д., но нам бы не удалось достичь результата ImageNet-конфигурации.

Выходы с предпоследнего слоя ImageNet, в свою очередь, малоразмерны (4096 в противовес нескольким сотням тысяч, как было в первой части), что позволяет эффективно использовать LinearSVC и сэкономить время на обучении. Более того, сеть уже обучена на большем количестве данных, чем мы когда-либо могли бы собрать; а представление изображений в виде векторов с предпоследнего слоя занимает уже немного времени.

Таким образом, в условии ограниченного объёма данных, временных (учитывая Runtime Google Colab :D) и аппаратных ресурсов, ImageNet + LinearSVM мне кажется наиболее перспективным для данной задачи из обозначенных подходов; его и будем улучшать.